In [1]:
#Import needed modules
import pandas as pd
import pymarc
import re
from pymarc import MARCWriter 
from pymarc import MARCReader
from pymarc import Record
from pymarc.field import Field
import csv

In [2]:
#Create pandas dataframe
MasterChart = pd.read_csv('MasterChartcleaner.csv')
#Index the Publication_Place column for various MARC writer options below.
idx = pd.Index(MasterChart.Publication_Place)

print(idx)

Index(['A Bordeaux', 'A Chaalons', 'A Cologne', 'A Geneve', 'A la Haye',
       'A Leyden', 'A Lyon', 'À Lyon', 'A Montbeliart', 'A Paris',
       ...
       'Wormatiae', 'Wratislauiae', 'Zaganae', 'Zagrabiae', 'Zaragoza',
       'Zu Basel', 'Zullichouiae', 'Zürych', 'Zutphaniae', 'Zwollae'],
      dtype='object', name='Publication_Place', length=1327)


In [3]:
#NEED TO DO MORE -- Create CSV filewriter for writing Country Code, 260/264, 752, and 001,
with open('ReportMARCwith752s.csv', 'a') as csvfile:
    goodfilewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
with open('ReportMARCalreadywith752s.csv', 'a') as csvfile:
    okayishfilewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
with open('ReportMARCwithno752s.csv', 'a') as csvfile:
    badfilewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)

In [4]:
#Create reader and writer
reader = MARCReader(open('Test-37-Records.mrc', 'rb'))
writergood = MARCWriter(open('MARCwith752s.dat', 'wb'))
writerokayish = MARCWriter(open('MARCalreadywith752s.dat', 'wb'))
writerbad = MARCWriter(open('MARCwithno752s.dat', 'wb'))

#Create some variables
field008 = "No 008"
field260a = "No 260"
clean260a = "No 260"
field264a = "No 264"
clean264a = "No 264"
field752a = "no value"
field752b = "no value"
field752d = "no value"
field752e = "no value"
field7524 = "no value"
field7521 = "no value"
field260and264counter = 0
field260acounter = 0
field264acounter = 0


#Start loop through the records in the list.
for record in reader:
    #Grab 008 from record and assign to variable.
    field008 = record['008']
    #Grab MARC Country Code and assign to variable and remove any whitespace for 2 character-only codes.
    countrycode = field008.data[15:18].rstrip(' ')
    #Check to see if there is already a 752 in the record.
    if record['752'] is None:
        #Check to see how many 260s and 264s there are.
        field260and264counter = record.get_fields('260','264')
        if len(field260and264counter) == 1:
            #Check to see if there is a 260.
            if record['260'] is not None:
                #Check to see how many a subfields there are
                field260acounter = record['260'].get_subfields('a')
                if len(field260acounter) == 1:
                    field260a = record['260']['a'].rstrip(' :')
                    clean260a = result = re.sub('[\]\[\)\(\;\,\:\?]','',field260a)
                    #NEED TO DO -- Consider looking at LDR printed textual vs. manuscript textual a vs t record, 
                    #for now, this mostly works.
                    #Assign subfields based off of 260 value.
                    field752e = "publication place."
                    field7524 = "http://id.loc.gov/vocabulary/relators/pup.html"
                    #This Filters the original MasterChart dataframe for MARC Country Code and Publication Place.
                    QueriedChart = MasterChart.query('CountryCode == @countrycode and Publication_Place == @clean260a', inplace = False)
                    #This returns the size of the QueriedChart, incase there aren't any matches.
                    size = QueriedChart.size 
                    #Used to send records to the Badwriter if False; Goodwriter if True.
                    IndexChecker = idx.contains(clean260a)
                #if there is more or less than 1 subfield a, send it to the Badwriter.
                else:
                    IndexChecker = False
            #Check to see if there is a 264.
            elif record['264'] is not None:
                #Check to see how many $a subfields there are
                field264acounter = record['264'].get_subfields('a')
                if len(field264acounter) == 1:
                    field264a = record['264']['a'].rstrip(' :')
                    clean264a = result = re.sub('[\]\[\)\(\;\,\:\?]','',field264a)
                    #This Filters the original MasterChart dataframe for MARC Country Code and Publication Place
                    QueriedChart = MasterChart.query('CountryCode == @countrycode and Publication_Place == @clean264a', inplace = False)
                    #This returns the size of the QueriedChart, incase there aren't any matches.
                    size = QueriedChart.size 
                    #Used to send records to the Badwriter if False; Goodwriter if True.
                    IndexChecker = idx.contains(clean264a)
                #if there is more or less than 1 subfield a, send it to the Badwriter.
                else:
                    IndexChecker = False
                #Assign value to 752 subfield e and subfield 4 based on 264 indicators.
                for imprintrole in record.get_fields('264'):
                    #Assign 752 subfield e and 4 for production place
                    if imprintrole.indicator2 == '0':
                        field752e = "production place."
                        field7524 = "http://id.loc.gov/vocabulary/relators/prp.html"
                    #Assign 752 subfield e and 4 for publication place    
                    elif imprintrole.indicator2 == '1':
                        field752e = "publication place."
                        field7524 = "http://id.loc.gov/vocabulary/relators/pup.html"
                    #Assign 752 subfield e and 4 for distribution place    
                    elif imprintroleindicator2 == '2':
                        field752e = "distribution place."
                        field7524 = "http://id.loc.gov/vocabulary/relators/dbp.html"
                    #Assign 752 subfield e and 4 for manufacture place  
                    elif imprintroleindicator2 == '3':
                        field752e = "manufacture place."
                        field7524 = "http://id.loc.gov/vocabulary/relators/mfp.html"
                    #Handle records that do not have indicators with 264, unlikely, but possible.   
                    else:  
                        #Sends problem records with 264s with no indicators to the Badwriter.
                        IndexChecker = False
            #Sends problem records with no 260/264 at all to the Badwriter.            
            else:  
                IndexChecker = False
        #if there is more or less than one 260/264 tag combo, send it to the Badwriter.
        else:
             IndexChecker = False
        #If the cleaned-up versions of the 260/264 values were found using the IndexChecker, aka the IndexChecker is True.
        if IndexChecker is True and size > 0:
            #Assign the value of Term3 which gives the index of the row in the MasterChart.
            Term3 = QueriedChart.index[0]
            #Assign the values of the 752 subfields that weren't already assigned above. 
            field752a = MasterChart.loc[Term3,"752a"]
            field752b = MasterChart.loc[Term3,"752b"]
            field752d = MasterChart.loc[Term3,"752d"]
            field7521 = MasterChart.loc[Term3,"Geonames_URI"]
            #Add the assembled 752 subfields to the MARC Record.   
            record.add_ordered_field(
            Field(
            tag = '752',
            indicators = [' ',' '],
            subfields = [
                'a', field752a,
                'b', field752b,
                'd', field752d,
                'e', field752e,
                '4', field7524,
                '1', field7521,
                    ]))
            #remove any subfields with "no value" as a subfield value.
            if field752a == 'no value':
                record['752'].delete_subfield('a')
            else:
                print(record['752'])
            if field752b == 'no value':
                record['752'].delete_subfield('b')
            else:
                print(record['752'])
            if field752d == 'no value':
                record['752'].delete_subfield('d')
            else:
                print(record['752'])
            if field752e == 'no value':
                record['752'].delete_subfield('e')
            else:
                print(record['752'])
            if field7521 == 'no value':
                record['752'].delete_subfield('1')
            else:
                print(record['752'])
         #The MARC record is written to the Goodwriter after the 752 is populated and added to the record.  
        #Create dictionary of values for Goodwriter CSV file
            MMSID = [record['001']]
            OCLCno = [record['035']]
            ccfrom008 = countrycode
            field260 = [record['260']]
            field264 = [record['264']]
            field752 = [record['752']]
            #Create a dictionary of lists 
            dict = {'Alma MMS ID': MMSID, 'OCLC number': OCLCno, 'Country Code': ccfrom008, '260': field260, '264': field264, '752': field752}
            gooddf = pd.DataFrame(dict) 
            #Saves MARC information to the good dataframe 
            gooddf.to_csv('ReportMarcwith752s.csv', mode='a', header=False) 
            writergood.write(record)          
        #If the cleaned-up versions of the 260/264 values were NOT found using the IndexChecker, aka the IndexChecker is False.
        #The MARC record is written to the Badwriter for manual review and evaluation.
        else:
            #Create dictionary of values for Badwriter CSV file
            MMSID = [record['001']]
            OCLCno = [record['035']]
            ccfrom008 = countrycode
            field260 = [record['260']] 
            field264 = [record['264']]
            field752 = [record['752']]
            #Create a dictionary of lists 
            dict = {'Alma MMS ID': MMSID, 'OCLC number': OCLCno, 'Country Code': ccfrom008, '260': field260, '264': field264, '752': field752}
            baddf = pd.DataFrame(dict) 
            #Saves MARC information to the bad dataframe 
            baddf.to_csv('ReportMARCwithno752s.csv', mode='a', header=False)  
            writerbad.write(record)
    #If there is already a 752 in the record, send to the Okayish writer for manual review.
    else:
        #Create dictionary of values for Okayish CSV file 
        MMSID = [record['001']]
        OCLCno = [record['035']]
        ccfrom008 = countrycode
        field260 = [record['260']] 
        field264 = [record['264']]
        field752 = [record['752']]
        #Create a dictionary of lists 
        dict = {'Alma MMS ID': MMSID, 'OCLC number': OCLCno, 'Country Code': ccfrom008, '260': field260, '264': field264, '752': field752}
        okayishdf = pd.DataFrame(dict) 
        #Saves MARC information to the okayish dataframe 
        okayishdf.to_csv('ReportMARCalreadywith752s.csv', mode='a', header=False) 
        writerokayish.write(record)
#Close both writers                   
writergood.close()      
writerbad.close()
writerokayish.close()
    
    

=752  \\$aFrance$bno value$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication pl

=752  \\$aNetherlands$bno value$dAmsterdam,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2759794
=752  \\$aNetherlands$dAmsterdam,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2759794
=752  \\$aNetherlands$dAmsterdam,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2759794
=752  \\$aNetherlands$dAmsterdam,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2759794
=752  \\$aNetherlands$bno value$dHague,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1https://sws.geonames.org/2747372
=752  \\$aNetherlands$dHague,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1https://sws.geonames.org/2747372
=752  \\$aNetherlands$dHague,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1https://sws.geonames.org/2747372
=752  \\$aNetherlands$dHague,$epublication p

=752  \\$aFrance$bno value$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aItaly$bno value$dVenice,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3164603
=752  \\$aItaly$dVenice,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3164603
=752  \\$aItaly$dVenice,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3164603
=752  \\$aItaly$dVenice,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http